###CLASSIFICATION

Modelo DNN que predice la probabilidad de de que una flor, dadas la longitud y anchura de pétalos y sépalos, pertenezca a una de las 3 especies del género Iris.

In [1]:
%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [3]:
# Nombre de la cabecera del dataset. Medidas de pétalos y sépalos.
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']

# Diferentes categorías, en este caso especies para las que se quiere predecir la pertenencia de las flores
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [4]:
# Usa el submódulo Keras, de TensorFlow para guardar el archivo csv con los nombres "iris_training.csv" y "iris_test.csv" en el runtime (en la nube)

# Dataset de entrenamiento
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")

# Dataset de evaluación/test
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

# Carga de los "dataframes" del módulo pandas (como en el ejemplo de regresión lineal)
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0) # header 0 indica que el nombre de cada columna, será la fila 0 del dataset (tabla)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train.head() 
# vistazo al dataframe, vemos que la cabecera tiene los nombres del array "CSV_COLUMN_NAMES" que declaramos anteriormete
# Las especies están codificadas en números enteros (los datos categóricos están ya tratados así que no es necesario clasificarlos)
# La anchura y longitud de los pétalos y sépalos está en cm.
# Las especies son:
# 0 : Setosa
# 1 : Versicolor
# 2 : Virginica

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
train.shape # vemos que se trata de 120 registros distribuídos en 5 columnas

(120, 5)

In [7]:
# Como hicimos en el modelo anterior. Del conjunto de datos, separamos el label. 
# Que representa la predicción. La especie que realmente tiene. 
# Que es lo que en última instancia se quiere predecir.
train_y = train.pop('Species') 
test_y = test.pop('Species')


In [ ]:
train.shape #ahora son 4 columnas
train_y.head() # muestra de cómo se guardan los valores del label

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

FEATURE COLUMNS


In [ ]:
# Instanciamos las feature columns, esta vez, todos los datos son numéricos así que es más sencillo.
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

print(my_feature_columns)


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


INPUT FUNCTION

In [ ]:
# definición de la input function, esta vez no tiene epochs
def input_fn (features, labels, training=True, batch_size=256):
  # Convierte los inputs en un Dataset 
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)


Building the model:DNNClassifier

In [ ]:
# Definición de un DNNClassifier 
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30,10], # 2 Hidden layers, de 30 y 10 nodos cada una
    n_classes=3) # 

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_mp6gk5w', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
# definición de una función lambda
x = lambda: print("hi")
x()

hi


TRAINING THE MODEL

In [ ]:
# Esto entrena el modelo directamente
# train es el dataframe de entrenamiento
# train_y es el label del mismo
# el uso de una función lambda nos permite introducir una función directamente como parámetro dentro de un método :D
classifier.train(
    input_fn=lambda: input_fn(train,train_y, training=True),
    steps=6000) # los modelos basados en DNN se basan en steps en lugar de epochs

In [ ]:
# Evaluamos el modelo
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False)) #en este caso le damos los datos de test y cambiamos training a false

# imprimimos la precisión del modelo.
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

# Prueba a cambiar el parámetro "steps", vuelve a entrenar el modelo y evalúalo
# A nosotros nos cambió de 0.7 a 0.93 al cambiar de 5000 a 6000 steps.
# Cabe destacar que cuando se evalúa el modelo, no hay steps como el entrenamiento. Se proporciona toda la información en una sola vez.

SCRIPT DE PREDICCIONES

In [ ]:
# Input function que le daremos al modelo para que nos de la respuesta.
# Vemos que no se da ningún "label" o Y value (respuestas), porque queremos que el modelo nos de la respuesta
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength','PetalWidth'] # especies
predict = {} # diccionario key:value

# Se le va pidiendo un valor al usuario por cada feature
# El valo se añade al diccionario haciendo un casting a float
print("Por favor introduzca los valores numéricos que se le piden.")
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

# Predicción 
# Ejecutamos una predicción dándole la input function que habíamos definido
# Le pasamos el diccionario que acabamos de crear con los input para cada parte anatómica de la flor
predictions = classifier.predict(input_fn=lambda: input_fn(predict))

# Obtenemos los valores que necesitamos de las predicciones
for pred_dict in predictions: # predictions tiene numerosos diccionarios
  print(pred_dict) # si imprimimos el diccionario de predicciones podemos ver que en 'class_ids' se encuentra el índice de la clase que tiene mayor probabilidad. Con este índice podemos sacar el resto de datos por pantalla. Si vamos arriba del todo vemos que 0,1,2 se corresponden con las especies que introducimos
  class_id = pred_dict['class_ids'][0] # id de la clase (0 porque el valor más probable se almacena en la posición cero)
  probability = pred_dict['probabilities'][class_id] # probabilidad de que sea esa clase

  # imprime con un formato, entre llaves los valores
  print('Prediction is "{}" ({:.1f}%)'.format( 
        SPECIES[class_id], 100 * probability))



Por favor introduzca los valores numéricos que se le piden.
SepalLength: 2.2
SepalWidth: 2.4
PetalLength: 5.6
PetalWidth: 5.3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp_mp6gk5w/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.0792955,  1.1768086,  4.5585027], dtype=float32), 'probabilities': array([0.009274  , 0.03256764, 0.95815843], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (95.8%)


In [ ]:
print(probability)

0.98263806


In [ ]:
# Algunos ejemplos para probar. Ejemplo: si introducimos 5.1, 3.3, 1.7 y 0.5 debería salir especie 'Setosa'
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}